# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:

**Prerequisites:**
-  00 - Environment Setup

**Overview:**
-  Setup BigQuery
   -  Create a Dataset
      -  Use BigQuery Python Client
   -  Create Tables
      -  Copy from another Project:Dataset
         -  SQL with BigQuery Jupyter Magic (%%bigquery)
      -  Load data from GCS Bucket
         -  BigQuery Python Client (load_table_from_uri)
   -  Prepare Data For Analysis
      -  Run SQL Queries to prepare Unique ID's and Train/Test Splits

**Resources:**
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Download BigQuery Data to Pandas](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)
-  [BigQuery Template Notebooks](https://github.com/GoogleCloudPlatform/bigquery-notebooks/tree/main/notebooks/official/template_notebooks)

**Related Training:**
-  todo


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/slide_05.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/slide_06.png">

---
## Source Data

**Overview**

This notebook imports source data for this project into Google BigQuery.  All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

In the enviornment setup notebook (00), a BigQuery source table was exported to CSV format in a Cloud Storage Bucket. This notebook, `01 - BigQuery - Table Data Source`, start the machine learning lifecycle by importing a source and preparing it for machine learning.  To customize this series of notebooks change the source reference here or in notebook `00 - Environment Setup`.

All of the workflows utlize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns.  The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.  

**The Data**

The source data is exported to Google Cloud Storage in CSV format in the `00 - Environment Setup` notebook.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.  

The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).

**Description of the Data**

This is a table of 284,207 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction

**Preparation of the Data**

This notebook adds two columns to the source data and stores it in a new table with suffix `_prepped`.  
- `transaction_id` (string) a unique id for the row/transaction
- `splits` (string) this divided the tranactions into sets for `TRAIN` (80%), `VALIDATA` (10%), and `TEST` (10%)

---
## Setup

inputs:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '01'

packages:

In [2]:
from google.cloud import bigquery

clients:

In [3]:
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [4]:
BUCKET = PROJECT_ID

---
## Create Dataset

List BigQuery datasets in the project:

In [5]:
query = f"""
SELECT schema_name
FROM `{PROJECT_ID}.INFORMATION_SCHEMA.SCHEMATA`
"""
bq.query(query = query).to_dataframe()

,schema_name


Create the dataset if missing:

In [6]:
query = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATANAME}`
OPTIONS(
    location = '{REGION}',
    labels = [('notebook','{NOTEBOOK}')]
)
"""
job = bq.query(query = query)
job.result()

In [7]:
(job.ended-job.started).total_seconds()

0.891

---
## Create Table
- import data from Cloud Storage Bucket
- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

In [8]:
destination = bigquery.TableReference.from_string(f"{PROJECT_ID}.{DATANAME}.{DATANAME}")
job_config = bigquery.LoadJobConfig(
    write_disposition = 'WRITE_TRUNCATE',
    source_format = bigquery.SourceFormat.CSV,
    autodetect = True,
    labels = {'notebook':f'{NOTEBOOK}'}
)
job = bq.load_table_from_uri(f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv", destination, job_config = job_config)
job.result()

In [9]:
(job.ended-job.started).total_seconds()

13.407

In [10]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}`
LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,380,-1.299837,0.881817,1.452842,-1.293698,-0.025105,-1.170103,0.861610,-0.193934,0.592001,...,-0.272563,-0.360853,0.223911,0.598930,-0.397705,0.637141,0.234872,0.021379,0.0,0
2,403,1.237413,0.512365,0.687746,1.693872,-0.236323,-0.650232,0.118066,-0.230545,-0.808523,...,-0.077543,-0.178220,0.038722,0.471218,0.289249,0.871803,-0.066884,0.012986,0.0,0
3,430,-1.860258,-0.629859,0.966570,0.844632,0.759983,-1.481173,-0.509681,0.540722,-0.733623,...,0.268028,0.125515,-0.225029,0.586664,-0.031598,0.570168,-0.043007,-0.223739,0.0,0
4,711,-0.431349,1.027694,2.670816,2.084787,-0.274567,0.286856,0.152110,0.200872,-0.596505,...,0.001241,0.154170,-0.141533,0.384610,-0.147132,-0.087100,0.101117,0.077944,0.0,0


---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [11]:
query = f"""
CREATE OR REPLACE TABLE `{DATANAME}.{DATANAME}_prepped` AS
SELECT *, GENERATE_UUID() transaction_id,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(GENERATE_UUID())),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(GENERATE_UUID())),10) < 5 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM `{DATANAME}.{DATANAME}`
"""
job = bq.query(query = query)
job.result()

In [12]:
(job.ended-job.started).total_seconds()

10.915

In [13]:
job.estimated_bytes_processed/1000000

70.632136

Review the test/train split:

In [14]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{DATANAME}.{DATANAME}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28308,9.939362
1,VALIDATE,28643,10.056986
2,TRAIN,227856,80.003652


Retrieve a subset of the data to a Pandas dataframe:

In [15]:
query = f"""
SELECT * 
FROM `{DATANAME}.{DATANAME}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [16]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,46100,0.971963,-0.064002,1.864457,2.521220,-0.700822,1.660426,-1.206327,0.717368,0.170900,...,-0.005019,-0.259129,0.152646,0.207666,0.093585,0.022994,0.0,0,3eddd943-117e-4ba9-a09b-c0e2fe8c5647,TEST
1,80430,-0.929700,0.194664,1.549227,1.693430,1.038639,-0.214545,-0.032843,0.248009,-0.598265,...,0.263542,-0.060427,-1.227930,-0.634669,0.062523,0.343741,0.0,0,8f25f7a0-63a0-4a7b-a0de-cdebe813ccee,TEST
2,123919,1.890586,0.271313,-0.157228,4.064907,-0.109914,0.150175,-0.230044,0.061367,-0.119159,...,0.092487,0.030498,0.074608,0.134964,-0.008043,-0.048927,0.0,0,a0da598e-391b-4b9b-a0ba-f8977d0b43b0,TEST
3,141191,-2.857621,-0.307727,1.521266,4.500119,1.812809,2.276221,-0.425395,0.895603,-1.564402,...,1.374592,-1.723268,0.127809,0.207510,-0.082130,-0.009999,0.0,0,7159db57-0ebd-4b99-a1b4-de6996ae0bd7,TEST
4,87426,-1.411512,1.067355,1.409569,4.292509,0.430227,2.831664,-0.748377,0.944136,-0.633843,...,0.252731,-1.528922,-1.117990,0.486644,-0.861374,0.089722,0.0,0,8a2be5ec-5687-4066-a878-d377ed0937ff,TEST


---
## Remove Resources
see notebook "XX - Cleanup"